In [ ]:
from ultralytics import YOLO
import json
import os
import shutil

# Paths
json_file_path = "../workspace/nuimages_output/train/labels.json"
image_folder = "../workspace/nuimages_output/train"
yolo_dataset_path = "yolo_dataset"
yolo_train_images_path = os.path.join(yolo_dataset_path, 'images', 'train')
yolo_train_labels_path = os.path.join(yolo_dataset_path, 'labels', 'train')
yolo_val_images_path = os.path.join(yolo_dataset_path, 'images', 'val')
yolo_val_labels_path = os.path.join(yolo_dataset_path, 'labels', 'val')

# Create directories
os.makedirs(yolo_train_images_path, exist_ok=True)
os.makedirs(yolo_train_labels_path, exist_ok=True)
os.makedirs(yolo_val_images_path, exist_ok=True)
os.makedirs(yolo_val_labels_path, exist_ok=True)

# Load the JSON file
with open(json_file_path, "r") as file:
    annotations = json.load(file)

# Print original annotations for debugging
print("Original Annotations (First 5):", annotations[:5])

# Define class mapping
class_mapping = {
    "car": ["automobile", "taxi", "vehicle", "suv", "jeep", "sedan", "van", "land vehicle", "vehicle.car", "vehicle.emergency.police"],
    "truck": ["truck", "lorry", "bus", "shuttle bus", "pickup truck", "vehicle.truck", "vehicle.bus.bendy", "vehicle.bus.rigid", "vehicle.trailer", "vehicle.construction", "vehicle.emergency.ambulance"],
    "person": ["person", "pedestrian", "human", "human.pedestrian.adult", "human.pedestrian.child", "human.pedestrian.construction_worker", "human.pedestrian.personal_mobility", "human.pedestrian.police_officer", "human.pedestrian.stroller", "human.pedestrian.wheelchair"],
    "biker": ["bicycle", "bike", "biker", "motorcycle", "motorbike", "vehicle.bicycle", "vehicle.motorcycle"],
}

# Function to convert class names to YOLO format
def convert_class_name(class_name):
    for key, values in class_mapping.items():
        if class_name in values:
            return key
    return None

# Filter and map annotations
filtered_annotations = []
for annotation in annotations:
    filtered_objects = []
    print(f"Processing annotation for image_id: {annotation['image_id']}")
    for obj in annotation["objects"]:
        class_name = obj["class_name"]
        new_class_name = convert_class_name(class_name)
        if new_class_name is not None:
            obj["class_name"] = new_class_name
            filtered_objects.append(obj)
        else:
            print(f"Class {class_name} not in class mapping. Skipping.")
    if filtered_objects:
        annotation["objects"] = filtered_objects
        filtered_annotations.append(annotation)

# Save annotations in YOLO format
for annotation in filtered_annotations:
    image_filename = os.path.basename(annotation["image_id"])  # Ensure we get only the filename
    image_path = os.path.join(image_folder, image_filename)

    if not os.path.exists(image_path):
        print(f"Image {image_path} does not exist. Skipping.")
        continue

    shutil.copy(image_path, os.path.join(yolo_train_images_path, image_filename))

    label_filename = os.path.splitext(image_filename)[0] + '.txt'
    label_path = os.path.join(yolo_train_labels_path, label_filename)
    with open(label_path, 'w') as label_file:
        for obj in annotation["objects"]:
            class_id = list(class_mapping.keys()).index(obj["class_name"])
            bbox = obj["bbox"]
            
            # Convert top-left to center coordinates
            center_x = bbox[0] + bbox[2] / 2
            center_y = bbox[1] + bbox[3] / 2
            
            label_file.write(f"{class_id} {center_x} {center_y} {bbox[2]} {bbox[3]}\n")

# Verify images are copied correctly before splitting
train_images = os.listdir(yolo_train_images_path)
print(f"Number of training images before split: {len(train_images)}")

# Split dataset into training and validation
val_split_ratio = 0.1
val_count = int(val_split_ratio * len(train_images))
val_images = train_images[:val_count]

for val_image in val_images:
    src_image_path = os.path.join(yolo_train_images_path, val_image)
    dst_image_path = os.path.join(yolo_val_images_path, val_image)
    src_label_path = os.path.join(yolo_train_labels_path, os.path.splitext(val_image)[0] + '.txt')
    dst_label_path = os.path.join(yolo_val_labels_path, os.path.splitext(val_image)[0] + '.txt')
    
    print(f"Moving {src_image_path} to {dst_image_path}")
    shutil.move(src_image_path, dst_image_path)
    print(f"Moving {src_label_path} to {dst_label_path}")
    shutil.move(src_label_path, dst_label_path)

# Verify images are copied correctly after splitting
train_images = os.listdir(yolo_train_images_path)
val_images = os.listdir(yolo_val_images_path)
print(f"Number of training images: {len(train_images)}")
print(f"Number of validation images: {len(val_images)}")

# Create the dataset configuration file
dataset_yaml_path = os.path.join(yolo_dataset_path, 'dataset.yaml')
with open(dataset_yaml_path, 'w') as yaml_file:
    yaml_file.write(f'train: {os.path.abspath(yolo_train_images_path)}\n')
    yaml_file.write(f'val: {os.path.abspath(yolo_val_images_path)}\n')
    yaml_file.write('\nnc: 4\n') 
    yaml_file.write('names: ["car", "truck", "person", "biker"]\n')

# Initialize the YOLO model with pretrained weights for fine-tuning
model = YOLO('yolov8s.pt')

# Train the model with adjusted parameters for better performance
model.train(
    data=dataset_yaml_path,
    epochs=100,
    imgsz=1600, 
)